In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob

In [ ]:
file_list = glob("./data/*.csv")
file_list

In [ ]:
mca = pd.read_csv(file_list[2], encoding="cp437", low_memory=False)
mca.shape

In [ ]:
pd.set_option("display.max_colwidth", None)

In [ ]:
mca["BlogsPodcastsNewslettersSelect"].head()

In [ ]:
pd.reset_option("display.max_colwidth")

In [ ]:
media_series = mca["BlogsPodcastsNewslettersSelect"].str.split(",", expand=True).stack().reset_index(level=1, drop=True)
media_series.shape

In [ ]:
media_count_series = media_series.value_counts().head(20)
media_count_series

In [ ]:
sns.set(font_scale=0.8, font="Malgun Gothic")

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(y=media_count_series.index, x=media_count_series.values)

In [ ]:
media_origin_series = mca["BlogsPodcastsNewslettersSelect"].value_counts().head(20)
media_origin_series

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(x=media_origin_series.values, y=media_origin_series.index)

In [ ]:
course_series = mca["CoursePlatformSelect"].str.split(",", expand=True).stack().reset_index(level=1, drop=True)
course_series.head()

In [ ]:
course_count_series = course_series.value_counts()
course_count_series

In [ ]:
plt.title("Most popular course platforms")
sns.barplot(y=course_count_series.index, x=course_count_series.values)

In [ ]:
skill_list = list(mca.columns[mca.columns.str.contains("^JobSkillImportance[^Other]", regex=True)])
skill_list

In [ ]:
mca["JobSkillImportanceBigData"].value_counts()

In [ ]:
skill_rate_dict = {}
for skill in skill_list:
    temp = mca[skill].value_counts(normalize=True)
    skill_rate_dict[skill[len("JobSkillImportance"):]] = temp
skill_rate_dict


In [ ]:
skill_rate_df = pd.DataFrame(skill_rate_dict).transpose()
skill_rate_df.head()

In [ ]:
sns.heatmap(skill_rate_df.sort_values("Necessary", ascending=False), annot=True, fmt=".2f", cmap="Blues")

In [ ]:
skill_rate_plot = skill_rate_df.plot.bar(figsize=(12, 6))
plt.xticks(rotation=60)
skill_rate_plot

In [ ]:
mca["CompensationAmount"].shape

In [ ]:
mca.replace({"CompensationAmount": {"[,-]": ""}}, regex=True, inplace=True)
mca["CompensationAmount"].str.contains("[,-]", regex=True, na=False).unique()

In [ ]:
currency_rate_df = pd.read_csv(file_list[0])
currency_rate_df.drop("Unnamed: 0", axis=1, inplace=True)
currency_rate_df

In [ ]:
salary_df = mca[["CompensationAmount", "CompensationCurrency", "GenderSelect", "Country", "Age", "CurrentJobTitleSelect"]].dropna().copy()
salary_df.head()

In [ ]:
salary_df = salary_df.merge(currency_rate_df, "left", left_on="CompensationCurrency", right_on="originCountry")
salary_df.head()

In [ ]:
salary_df.info()

In [ ]:
salary_df.isnull().sum()

In [ ]:
salary_df["CompensationAmount"].replace({"": 0}, inplace=True)

In [ ]:
salary_df["salary"] = pd.to_numeric(salary_df["CompensationAmount"]) * salary_df["exchangeRate"]
salary_df.head()

In [ ]:
# salary_df["salary"].describe()
print(f"Maximum Salary is USD $ {salary_df['salary'].max():.0f}",
      f"Minimum Salary is USD $ {salary_df['salary'].min():.0f}",
      f"Median Salary is USD $ {salary_df['salary'].median():.0f}", sep="\n")

In [ ]:
plt.figure(figsize=(12, 6))
salary_df = salary_df[salary_df["salary"] <= 500000]
sns.histplot(salary_df, 
             x="salary", 
             kde=True, 
             bins=100)
plt.axvline(salary_df["salary"].median(), linestyle="dashed")
plt.xticks(rotation= 30)
plt.title("Salary Distribution")

In [ ]:
sal_cnt_df = salary_df \
    .groupby(["Country"])["salary"] \
    .median() \
    .sort_values(ascending=False) \
    [:30] \
    .to_frame()
sal_cnt_df.head()

In [ ]:
plt.figure(figsize=(8, 10))
sns.barplot(sal_cnt_df, 
            y=sal_cnt_df.index, 
            x="salary",
            palette="Reds_r")
plt.axvline(salary_df["salary"].median(), linestyle="dashed")
plt.title("Highest Salary Paying Countries")

In [ ]:
plt.figure(figsize=(8, 6))
plt.xticks(rotation=30)
sns.boxplot(salary_df,
            x="salary",
            y="GenderSelect")

In [ ]:
salary_df.columns

In [ ]:
salary_korea_df = salary_df[salary_df["Country"] == "South Korea"].copy()
salary_korea_df

In [ ]:
salary_korea_df["Age"].describe()

In [ ]:
plt.axvline(salary_korea_df["Age"].median(), linestyle="dashed")
sns.histplot(salary_korea_df, 
             x="Age", 
             kde=True)

In [ ]:
sns.lmplot(salary_korea_df,
            x="Age",
            y="salary",
            hue="GenderSelect"
            )

In [ ]:
plt.figure(figsize=(8, 3))
sns.boxplot(salary_korea_df, 
            y="GenderSelect", 
            x="salary")

In [ ]:
salary_korea_df["GenderSelect"].value_counts()

In [ ]:
salary_korea_df[salary_korea_df["GenderSelect"].str.contains("Female")]

In [ ]:
salary_korea_df[salary_korea_df["GenderSelect"].str.contains("Male")]["salary"].describe()

In [ ]:
salary_korea_df[salary_korea_df["GenderSelect"].str.contains("Male")]

In [ ]:
pd.set_option("display.max_colwidth", None)

In [ ]:
dataset_series = mca["PublicDatasetsSelect"].str.split(",", expand=True).stack().reset_index(level=1, drop=True)
dataset_series.name = "dataset"
dataset_series.head()

In [ ]:
dataset_count_series =dataset_series.value_counts()
dataset_count_series

In [ ]:
dataset_count_series.index

In [ ]:
dataset_count_series.values

In [ ]:
sns.barplot(y=dataset_count_series.index,
            x=dataset_count_series.values)

In [ ]:
file_list

In [ ]:
# ffa
ffa = pd.read_csv(file_list[1], low_memory=False)
ffa.shape

In [ ]:
mcq = pd.read_csv(file_list[3])

In [ ]:
mcq.columns

In [ ]:
mcq.loc[mcq["Column"].str.contains("PersonalProjectsChallengeFreeForm"), "Question"]

In [ ]:
ffa["PersonalProjectsChallengeFreeForm"].notnull().sum()

In [ ]:
ffa["PersonalProjectsChallengeFreeForm"].value_counts(dropna=True).head(15)

In [ ]:
ffa[ffa["PersonalProjectsChallengeFreeForm"].str.contains("fusion", na=False, regex=True)]

In [ ]:
time_list = mcq[(mcq["Column"].str.contains("^Time", regex=True)) 
                & (mcq["Asked"] == "CodingWorker")
                & ~(mcq["Column"].str.contains("FreeForm$", regex=True))]["Column"].tolist()
time_list

In [ ]:
plt.axvline(mca[time_list[0]].median(), linestyle="dashed")
plt.axvline(mca[time_list[0]].mean())
sns.histplot(mca,
             x=mca[time_list[0]],
             kde=True)

In [ ]:
plt.figure(figsize=(1, 5))
sns.boxplot(mca,
            y=mca[time_list[0]].dropna(),
            palette="Blues")
# sns.swarmplot(mca[time_list[0]].dropna(),
#             y=mca[time_list[0]].dropna(),
#             palette="Reds")

In [ ]:
time_dict = {}
for time in time_list:
    temp = mca[time].mean()
    time_dict[time[len("Time"):]] = temp

time_dict

In [ ]:
time_series = pd.Series(time_dict)
time_series

In [ ]:
time_series.plot.pie(autopct="%.2f%%")